In [1]:
import re
import os
import pickle
import numpy as np
from torch.utils.data import Dataset
from utils.utils import newsample, getId2idx, tokenize, getVocab, my_collate, Partition_Sampler
from data.configs.demo import config
from torch.utils.data import DataLoader

In [2]:
from utils.MIND import MIND

In [3]:
config.signal_length = 20
config.his_size = 10
train_path = config.path + 'MIND/MINDdemo_dev/'
a = MIND(config, train_path + 'news.tsv', train_path + 'behaviors.tsv')

[2021-08-16 17:21:33,971] INFO (utils.MIND) encoding news and behaviors...


In [4]:
import pickle
news = pickle.load(open('data/cache/bert/MINDdemo_dev/10/news.pkl','rb'))

In [6]:
news['encoded_news'].shape

(42417, 512)

In [21]:
from transformers import BertTokenizer

t = BertTokenizer.from_pretrained('bert-base-uncased',cache=config.path + 'bert_cache/')

In [32]:
t('[CLS] I love you', add_special_tokens=False)

{'input_ids': [101, 1045, 2293, 2017], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]}

In [29]:
ma = {
    'unk_token': '[UNK]',
    'pad_token': '[PAD]',
    'cls_token': '[CLS]'
}

In [30]:
t.special_tokens_map = ma

AttributeError: can't set attribute

In [4]:
s1 = Partition_Sampler(a, num_replicas=3, rank=0)
s2 = Partition_Sampler(a, num_replicas=3, rank=1)
s3 = Partition_Sampler(a, num_replicas=3, rank=2)
len(s3)

686

685

In [7]:
loader1 = DataLoader(a, batch_size=1, pin_memory=False, num_workers=0, drop_last=False, shuffle=False, sampler=s1)
records1 = list(loader1)

KeyError: 'label'

In [23]:
import torch

a = torch.load('/home/peitian_zhang/Codes/Document-Reduction/Code/data/model_params/esm__cnn__rnn-u__matching-based__2dcnn/demo_epoch1_step147_[k=3].model')

In [26]:
a['model'].keys()

odict_keys(['module.embedding.embedding.weight', 'module.encoderN.query_words', 'module.encoderN.wordQueryProject.weight', 'module.encoderN.wordQueryProject.bias', 'module.encoderN.CNN.weight', 'module.encoderN.CNN.bias', 'module.encoderN.layerNorm.weight', 'module.encoderN.layerNorm.bias', 'module.encoderU.lstm.weight_ih_l0', 'module.encoderU.lstm.weight_hh_l0', 'module.encoderU.lstm.bias_ih_l0', 'module.encoderU.lstm.bias_hh_l0', 'module.interactor.SeqCNN2D.0.weight', 'module.interactor.SeqCNN2D.0.bias', 'module.interactor.SeqCNN2D.3.weight', 'module.interactor.SeqCNN2D.3.bias', 'module.learningToRank.0.weight', 'module.learningToRank.0.bias', 'module.learningToRank.2.weight', 'module.learningToRank.2.bias'])

In [31]:
from typing import OrderedDict
model_dict = OrderedDict()

pattern = re.compile('module.')

for k,v in a['model'].items():
    if re.search("module", k):
        model_dict[re.sub(pattern, '', k)] = v
    else:
        break

In [30]:
model_dict

OrderedDict()

In [18]:
a = {'a':1}


In [19]:
a['b'] = a['a']
del a['a']

In [22]:
import re
c = re.compile('module.')
re.sub(c,'','module.embedding.weight')

'embedding.weight'

In [13]:
loader2 = DataLoader(a, batch_size=1, pin_memory=False, num_workers=0, drop_last=False, shuffle=False, sampler=s2)
records2 = list(loader2)

In [15]:
len(a)

2056

In [19]:
len(loader1)*3

2055

In [ ]:
record['cdd_attn_mask'].shape, record['cdd_encoded_index'].shape